In [13]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import xml.etree.ElementTree as ET
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from PIL import Image
from utils import IoU
from eval import iou
from detection_datasets import VOCDatasetV2

In [14]:
import pickle
with open('anchors_VOC0712trainval.pickle', 'rb') as handle:
    anchors = pickle.load(handle)

In [15]:
transforms = A.Compose([
    A.Resize(width=224, height=224),
    A.VerticalFlip(p=1.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))
train_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', scales=[80, 40, 20], anchors=anchors, transforms=transforms)
image, gt_out = train_set[0]

True ../../datasets/VOCdevkit/VOC2007\ImageSets\Main\trainval.txt
True ../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt


In [16]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)

In [17]:
for img in train_loader:
    del img

KeyboardInterrupt: 

In [27]:
train_set.object_placed, train_set.object_not_placed, train_set.object_placed + train_set.object_not_placed, train_set.total

(85646, 56038, 141684, 47228)

In [202]:
transforms = A.Compose([
    A.Resize(width=224, height=224),
    A.VerticalFlip(p=1.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))
train_set = VOCDataset(devkit_path = '../../datasets/VOCdevkit/', scales=[13], anchors=anchors, transforms=transforms)
image, gt_out = train_set[0]

True ../../datasets/VOCdevkit/VOC2007\ImageSets\Main\trainval.txt
True ../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt


In [203]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)

In [204]:
for img in train_loader:
    del img

In [205]:
train_set.object_placed, train_set.object_not_placed, train_set.object_placed + train_set.object_not_placed

(45658, 1570, 47228)